# <span style="font-family:'Times New Roman'"> IMPORT LIBRARIES</span>

In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# <span style="font-family: 'Times New Roman'">IMPORT DATA</span>

In [ ]:
movies = pd.read_csv('/kaggle/input/movies-and-ratings-for-recommendation-system/movies.csv')
movies.head()

In [ ]:
ratings = pd.read_csv('/kaggle/input/movies-and-ratings-for-recommendation-system/ratings.csv')
ratings.head()

# <span style="font-family:'Times New Roman'">PREPARE DATA</span>

In [ ]:
n_ratings = len(ratings)
n_movies = len(ratings['movieId'].unique())
n_users = len(ratings['userId'].unique())

print(f"Number of ratings: {n_ratings}")
print(f"Number of movies: {n_movies}")
print(f"Number of users: {n_users}")
print(f"Average ratings per user: {round(n_ratings/n_users, 2)}")
print(f"Average ratings per movie: {round(n_ratings/n_movies,2)}")

In [ ]:
user_freq = ratings[['userId', 'movieId']].groupby('userId').count().reset_index()
user_freq.columns = ['userId', 'n_ratings']
user_freq.head()

In [ ]:
# Find lowest and highest rated movies
mean_rating = ratings.groupby('movieId')[['rating']].mean()
# Lowest rated movies
lowest_rated = mean_rating['rating'].idxmin()
movies.loc[movies['movieId'] == lowest_rated]
# Highest rated movies
highest_rated = mean_rating['rating'].idxmax()
movies.loc[movies['movieId'] == highest_rated]

ratings[ratings['movieId'] == highest_rated]
ratings[ratings['movieId'] == lowest_rated]

movie_stats = ratings.groupby('movieId')[['rating']].agg(['count', 'mean'])
movie_stats.columns = movie_stats.columns.droplevel()


# <span style="font-family:'Times New Roman'">MODEL DEVELOPMENT</span>

In [ ]:
# Create user-item matrix using scipy csr matrix
def create_matrix(df):
    N = len(df['userId'].unique())
    M = len(df['movieId'].unique())
    
    # Map Ids to indices
    user_mapper = dict(zip(np.unique(df['userId']), list(range(N))))
    movie_mapper = dict(zip(np.unique(df['movieId']), list(range(M))))
                        
    # Map indices to Ids
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df['userId'])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df['movieId'])))
    
    user_index = [user_mapper[i] for i in df['userId']]
    movie_index = [movie_mapper[i] for i in df['movieId']]
    
    X = csr_matrix((df['rating'], (movie_index, user_index)), shape=(M, N))
                        
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper
                        
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_matrix(ratings)

In [ ]:
# Find similar movies using KNN
def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
    neighbor_ids = []
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_id]
    k+=1
    knn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric=metric)
    knn.fit(X)
    movie_vec = movie_vec.reshape(1, -1)
    neighbor = knn.kneighbors(movie_vec, return_distance=show_distance)
    for i in range(0,k):
        n = neighbor.item(i)
        neighbor_ids.append(movie_inv_mapper[n])
    neighbor_ids.pop(0)
    return neighbor_ids

movie_titles = dict(zip(movies['movieId'], movies['title']))
movie_id=7
similar_ids = find_similar_movies(movie_id, X, k=10)
movie_title = movie_titles[movie_id]
print(f"\033[1mSince you watched {movie_title}\033[0m")
for i in similar_ids:
    print(movie_titles[i])